In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import requests

In [4]:
# 그래프에 한글 사용 설정
from matplotlib import font_manager, rc
font_path = 'c:/Windows/Fonts/NGULIM.ttf'  # 폰트가 설치되어 있는 경로
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [5]:
import warnings
# 모든 경고 무시
warnings.filterwarnings('ignore')

#### 2024년 서울 인구 수 파일 불러오기

In [70]:
# 2024년 서울 인구 수 파일 불러오기
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_excel('서울인구.xlsx')
data.head()

,동별(1),동별(2),2024 3/4,2024 3/4.1,2024 3/4.2,2024 3/4.3,2024 3/4.4,2024 3/4.5,2024 3/4.6,2024 3/4.7,2024 3/4.8,2024 3/4.9,2024 3/4.10,2024 3/4.11
0,동별(1),동별(2),세대 (세대),계 (명),계 (명),계 (명),한국인 (명),한국인 (명),한국인 (명),등록외국인 (명),등록외국인 (명),등록외국인 (명),세대당인구 (명),65세이상고령자 (명)
1,동별(1),동별(2),소계,소계,남자,여자,소계,남자,여자,소계,남자,여자,소계,소계
2,합계,소계,4487671,9605419,4625980,4979439,9350995,4515791,4835204,254424,110189,144235,2.08,1803910
3,NaN,종로구,72341,150011,71561,78450,138879,66846,72033,11132,4715,6417,1.92,29598
4,NaN,중구,65378,131589,63248,68341,121324,58447,62877,10265,4801,5464,1.86,26712


In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   동별(1)        3 non-null      object
 1   동별(2)        28 non-null     object
 2   2024 3/4     28 non-null     object
 3   2024 3/4.1   28 non-null     object
 4   2024 3/4.2   28 non-null     object
 5   2024 3/4.3   28 non-null     object
 6   2024 3/4.4   28 non-null     object
 7   2024 3/4.5   28 non-null     object
 8   2024 3/4.6   28 non-null     object
 9   2024 3/4.7   28 non-null     object
 10  2024 3/4.8   28 non-null     object
 11  2024 3/4.9   28 non-null     object
 12  2024 3/4.10  28 non-null     object
 13  2024 3/4.11  28 non-null     object
dtypes: object(14)
memory usage: 3.2+ KB


In [72]:
# '동별(1)','2024 3/4' 열 삭제
# data.drop(['동별(1)', '2024 3/4'], axis = 1)
data.drop(columns = ['동별(1)', '2024 3/4'], inplace = True)
data.head()

,동별(2),2024 3/4.1,2024 3/4.2,2024 3/4.3,2024 3/4.4,2024 3/4.5,2024 3/4.6,2024 3/4.7,2024 3/4.8,2024 3/4.9,2024 3/4.10,2024 3/4.11
0,동별(2),계 (명),계 (명),계 (명),한국인 (명),한국인 (명),한국인 (명),등록외국인 (명),등록외국인 (명),등록외국인 (명),세대당인구 (명),65세이상고령자 (명)
1,동별(2),소계,남자,여자,소계,남자,여자,소계,남자,여자,소계,소계
2,소계,9605419,4625980,4979439,9350995,4515791,4835204,254424,110189,144235,2.08,1803910
3,종로구,150011,71561,78450,138879,66846,72033,11132,4715,6417,1.92,29598
4,중구,131589,63248,68341,121324,58447,62877,10265,4801,5464,1.86,26712


- 서울시 전체 인구 : 전체, 남자, 여자 인구 

In [73]:
# 서울시 전체 인구 : 전체, 남자, 여자 인구
data_total = data.iloc[:, 0:4].head(8)
data_total

,동별(2),2024 3/4.1,2024 3/4.2,2024 3/4.3
0,동별(2),계 (명),계 (명),계 (명)
1,동별(2),소계,남자,여자
2,소계,9605419,4625980,4979439
3,종로구,150011,71561,78450
4,중구,131589,63248,68341
5,용산구,218370,105220,113150
6,성동구,282385,136194,146191
7,광진구,349307,166408,182899


In [74]:
# 새로운 컬럼명을 설정하고, 0행 ~ 2행 삭제
data_total.columns = ['구', '소계', '남자', '여자']
data_total = data_total.drop(index = [0, 1, 2])
data_total

,구,소계,남자,여자
3,종로구,150011,71561,78450
4,중구,131589,63248,68341
5,용산구,218370,105220,113150
6,성동구,282385,136194,146191
7,광진구,349307,166408,182899


In [75]:
# 인덱스 리셋 
data_total.reset_index(drop = True, inplace = True)
data_total.head()

,구,소계,남자,여자
0,종로구,150011,71561,78450
1,중구,131589,63248,68341
2,용산구,218370,105220,113150
3,성동구,282385,136194,146191
4,광진구,349307,166408,182899


In [77]:
data_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구       5 non-null      object
 1   소계      5 non-null      object
 2   남자      5 non-null      object
 3   여자      5 non-null      object
dtypes: object(4)
memory usage: 292.0+ bytes


In [78]:
# '구' 열을 인덱스로 지정
data_total.set_index('구', inplace = True)
data_total

,소계,남자,여자
구,,,
종로구,150011,71561,78450
중구,131589,63248,68341
용산구,218370,105220,113150
성동구,282385,136194,146191
광진구,349307,166408,182899


In [79]:
# 모든 열을 int64로 변환
data_total = data_total.astype('int64')
data_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 종로구 to 광진구
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   소계      5 non-null      int64
 1   남자      5 non-null      int64
 2   여자      5 non-null      int64
dtypes: int64(3)
memory usage: 160.0+ bytes


In [80]:
data_total.head()

,소계,남자,여자
구,,,
종로구,150011,71561,78450
중구,131589,63248,68341
용산구,218370,105220,113150
성동구,282385,136194,146191
광진구,349307,166408,182899


- 서울시 한국인 인구 수 :전체, 남자, 여자 인구

In [81]:
# 서울시 한국인 인구 수 :전체, 남자, 여자 인구
data_ko = data.iloc[:, [0, 4, 5, 6]]
data_ko.head()

,동별(2),2024 3/4.4,2024 3/4.5,2024 3/4.6
0,동별(2),한국인 (명),한국인 (명),한국인 (명)
1,동별(2),소계,남자,여자
2,소계,9350995,4515791,4835204
3,종로구,138879,66846,72033
4,중구,121324,58447,62877


In [18]:
# 새로운 컬럼명을 설정하고, 0행 ~ 2행 삭제


# 인덱스 리셋 


,구,소계,남자,여자
0,종로구,138879,66846,72033
1,중구,121324,58447,62877
2,용산구,205145,98118,107027
3,성동구,275403,133271,142132
4,광진구,333403,159948,173455


In [19]:
data_ko.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구       25 non-null     object
 1   소계      25 non-null     object
 2   남자      25 non-null     object
 3   여자      25 non-null     object
dtypes: object(4)
memory usage: 932.0+ bytes


In [20]:
# '구' 열을 인덱스로 지정


In [21]:
# 모든 열을 int64로 변환


<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 종로구 to 강동구
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   소계      25 non-null     int64
 1   남자      25 non-null     int64
 2   여자      25 non-null     int64
dtypes: int64(3)
memory usage: 800.0+ bytes


In [22]:
data_ko.head()

,소계,남자,여자
구,,,
종로구,138879,66846,72033
중구,121324,58447,62877
용산구,205145,98118,107027
성동구,275403,133271,142132
광진구,333403,159948,173455


- 서울시 외국인 인구 수 :전체, 남자, 여자 인구

In [82]:
# 서울시 외국인 인구 수 :전체, 남자, 여자 인구
data_f = data.iloc[:, [0, 7, 8, 9]]
data_f.head()

,동별(2),2024 3/4.7,2024 3/4.8,2024 3/4.9
0,동별(2),등록외국인 (명),등록외국인 (명),등록외국인 (명)
1,동별(2),소계,남자,여자
2,소계,254424,110189,144235
3,종로구,11132,4715,6417
4,중구,10265,4801,5464


In [25]:
# 새로운 컬럼명을 설정하고, 0행 ~ 2행 삭제


# 인덱스 리셋 


,구,소계,남자,여자
0,종로구,11132,4715,6417
1,중구,10265,4801,5464
2,용산구,13225,7102,6123
3,성동구,6982,2923,4059
4,광진구,15904,6460,9444


In [26]:
data_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구       25 non-null     object
 1   소계      25 non-null     object
 2   남자      25 non-null     object
 3   여자      25 non-null     object
dtypes: object(4)
memory usage: 932.0+ bytes


In [27]:
# '구' 열을 인덱스로 지정


In [28]:
# 모든 열을 int64로 변환
data_f = data_ko.astype('int64')
data_f.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 종로구 to 강동구
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   소계      25 non-null     int64
 1   남자      25 non-null     int64
 2   여자      25 non-null     int64
dtypes: int64(3)
memory usage: 800.0+ bytes


In [29]:
data_f.head()

,소계,남자,여자
구,,,
종로구,138879,66846,72033
중구,121324,58447,62877
용산구,205145,98118,107027
성동구,275403,133271,142132
광진구,333403,159948,173455


In [83]:
# 완성된 데이터프레임 저장
# .csv 파일로 저장하기 : 인덱스 무시하고 저장
data_total.to_csv('seoul_total.csv')

In [64]:
# 완성된 데이터프레임 저장
# .csv 파일로 저장하기 : 인덱스 무시하고 저장
data_ko.to_csv('seoul_ko.csv')

In [66]:
# 완성된 데이터프레임 저장
# .csv 파일로 저장하기 : 인덱스 무시하고 저장
data_f.to_csv('seoul_f.csv')